In [2]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier 
from sklearn.preprocessing import LabelEncoder
import joblib

# 1. CARGA Y FILTRADO TRIPLE A (Top 10)
df = pd.read_csv("Resultados_de_Análisis_de_Laboratorio_Suelos_en_Colombia_20260216.csv", low_memory=False)
top_10 = df['Cultivo'].value_counts().nlargest(10).index
df_pro = df[df['Cultivo'].isin(top_10)].copy()

# 2. SELECCIÓN DE ACTIVOS QUÍMICOS
cols = ['Departamento', 'Municipio', 'Cultivo', 'pH agua:suelo', 'Materia organica', 
        'Fósforo Bray II', 'Potasio intercambiable', 'Calcio intercambiable', 
        'Magnesio intercambiable', 'Aluminio intercambiable']
df_pro = df_pro[cols]

# 3. CONVERSIÓN Y LIMPIEZA
for col in ['pH agua:suelo', 'Materia organica', 'Fósforo Bray II', 'Potasio intercambiable', 
            'Calcio intercambiable', 'Magnesio intercambiable', 'Aluminio intercambiable']:
    df_pro[col] = pd.to_numeric(df_pro[col], errors='coerce')
df_pro = df_pro.dropna()

# 4. INGENIERÍA DE RATIOS (Apalancamiento de Datos)
# Creamos variables inteligentes que el modelo amará
df_pro['Ratio_Ca_Mg'] = df_pro['Calcio intercambiable'] / (df_pro['Magnesio intercambiable'] + 0.1)
df_pro['Ratio_P_K'] = df_pro['Fósforo Bray II'] / (df_pro['Potasio intercambiable'] + 0.1)

# 5. CODIFICACIÓN
le_dept = LabelEncoder()
le_muni = LabelEncoder()
le_cultivo = LabelEncoder()

df_pro['dept_enc'] = le_dept.fit_transform(df_pro['Departamento'])
df_pro['muni_enc'] = le_muni.fit_transform(df_pro['Municipio'])
df_pro['cultivo_enc'] = le_cultivo.fit_transform(df_pro['Cultivo'])

# 6. PREPARACIÓN PARA EL DESPLIEGUE
X = df_pro[['dept_enc', 'muni_enc', 'pH agua:suelo', 'Materia organica', 'Fósforo Bray II', 
            'Potasio intercambiable', 'Calcio intercambiable', 'Magnesio intercambiable', 
            'Aluminio intercambiable', 'Ratio_Ca_Mg', 'Ratio_P_K']]
y = df_pro['cultivo_enc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. EL ML XGBOOST (Configuración de Competición)
print("Preparando el motor XGBoost...")
modelo_final = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method='hist' # Optimización de velocidad
)

modelo_final.fit(X_train, y_train)

# 8. EL MOMENTO DE LA RIQUEZA
precision = modelo_final.score(X_test, y_test)
print(f"¡PRECISIÓN DE ÉLITE ALCANZADA: {precision * 100:.2f}%!")

# Guardar el activo final
joblib.dump(modelo_final, 'modelo_colombia_70.pkl')

🏎️ Lanzando el Ferrari XGBoost al mercado colombiano...
💎 ¡PRECISIÓN DE ÉLITE ALCANZADA: 70.20%!


['modelo_colombia_70.pkl']